# 歡迎光臨
與舊版本不同，此版本是為了增加彈性而生。<br>
***請注意，使用此檔案前須將所有必要的附屬檔案放到這份Code存在的資料夾中。***
1. 在Vivado中，Generate Bitstream結束後產生的檔案：<br>
    ***(兩個檔名必須改成一樣)***
    * `.hwh` 檔案，位於 `{ project_location }/{project_name}.runs/impl_1/design_1_wrapper.bit`<br>
    * `.bit` 檔案，位於 `{ project_location }/{project_name}.gen/{Design Name 1}/bd/{Design Name 2}/hw_handoff/design_1.hwh`<br><br>
2. 原先Testbench所需的測試檔 (wip)
---
## 連結
點選連結快速跳到你想看的地方
* ### [參數設定](#設定預設值) 
* ### [函數定義](#函數定義) 
* ### [主程式起點](#主程式) 
---

* ### 設定基礎路徑

In [ ]:
baseDir = '/home/xilinx/jupyter_notebooks/[TOP] Flexible University Project/'

* ### 匯入所需模組

In [103]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO

import time

import ctypes
from ctypes import cdll, c_int32, c_uint, POINTER, c_char_p, c_int, c_float

import numpy as np

import sys
np.set_printoptions(threshold=sys.maxsize)

* #### 匯入C code dll

In [59]:
# 載入 SO 動態連結檔案
ifmaps_premodeling_dll = cdll.LoadLibrary(baseDir + 'ifmaps_premodeling.so')

# 設定 SO 檔案中 sort 函數的參數資料型態
ifmaps_premodeling_dll.pre_modeling.argtypes = [POINTER(c_uint), c_uint, c_uint, c_uint, c_uint, c_uint]
ifmaps_premodeling_dll.free_mem.argtypes = [POINTER(c_uint)]
# 設定 SO 檔案中 sort 函數的傳回值資料型態
ifmaps_premodeling_dll.pre_modeling.restype = POINTER(c_uint)
ifmaps_premodeling_dll.free_mem.restype = None


# 載入 SO 動態連結檔案
arr_axis_modeling_dll = ctypes.CDLL('baseDir' + 'arr_axis_modeling.so')

# 設定 SO 檔案中 sort 函數的參數資料型態
arr_axis_modeling_dll.arr_axis_modeling.argtypes = [POINTER(c_uint), c_uint, c_uint, c_uint, c_uint, c_uint]
arr_axis_modeling_dll.arr_axis_fc_modeling.argtypes = [POINTER(c_uint), c_uint, c_uint, c_uint, c_uint, POINTER(c_float), POINTER(c_float)]
arr_axis_modeling_dll.free_mem.argtypes = [POINTER(c_uint)]

# 設定 SO 檔案中 sort 函數的傳回值資料型態
arr_axis_modeling_dll.arr_axis_modeling.restype = POINTER(c_uint)
arr_axis_modeling_dll.arr_axis_fc_modeling.restype = c_int
arr_axis_modeling_dll.free_mem.restype = None


* ### memory test

In [60]:
# def how_many_free_mem():
#     mem = !cat /proc/meminfo | grep 'MemFree'
#     print(mem)

In [61]:
# how_many_free_mem()

* ### OOP-Wire
用法：
* a = wire(name = '', width = 3, data = 1)
* 可以用slice，會回傳boolean array (ex. a[1:3])
* 直接修改Data，如a.data = 12
* 可使用cascade疊加函數組合埠(底下有範例用法)

In [62]:
import math


class Wire:
    def __init__(self, width: int = 32, data: int = 0) -> None:
        # if len(args) == 1:
        #     # Check
        #     for _ in args:
        #         if not len(_) == 2 and not type(_) == list:
        #             raise TypeError('請注意輸入格式')
        # else:
        #     raise NotImplementedError('不支援同時綁定多條線')
        if not type(width) == int and not type(data) == int:
            raise TypeError('請輸入整數')
        if width < 1 and width % 1 == 0:
            raise ValueError('寬度須為正整數')
        self._OverflowChecker(width=width, data=data)
        self._width = width
        self._data = data

    @property
    def data(self):
        return self._data

    @property
    def width(self):
        return self._width

    @data.setter
    def data(self, newVal):
        self._OverflowChecker(width=self._width, data=newVal)
        self._data = newVal

    def __getitem__(self, idx):
        # Just access like Verilog
        l = self._IntToBoolConv(data=self._data, width=self._width)
        #l_len = len(l)
        # print('int2bool', l)
        # print(self)
        if type(idx) == int:
            return Wire(data=l[idx], width=1)
        else:
            l = l[idx.stop : idx.start + 1: idx.step]
            # print(l)
            if len(l) == 0:
                raise ValueError('陣列取值Start, Stop的順序錯誤！')
            # print('bool2int', self._bool2int(l))
            return Wire(data=self._bool2int(l), width=len(l))

    # Bitwise Setter
    def __setitem__(self, idx: int, newVal: bool):
        data = self._IntToBoolConv()
        if type(idx) == int:
            data[idx] = newVal
        else: 
            w = len(data[idx.stop:idx.start + 1:idx.step])
            total_l = len(data)
            data[idx.stop:idx.start+1:idx.step] = self._IntToBoolConv(width=w, data=newVal)
            #data[idx] = self._IntToBoolConv(width=len(data[idx]), data=newVal)

        self._data = self._bool2int(data)

    def __repr__(self) -> str:
        bin_arr=self._IntToBoolConv()
        bin_arr.reverse()

        return "[{}:0] = {} > {}".format(self._width - 1, self._data, list(map(int, (bin_arr))))

    def __len__(self):
        return self._width

    def _OverflowChecker(self, width: int = None, data: int = None) -> None:
        if data is None or width is None:
            data = self._data
            width = self._width
        if math.log2(data + 1) > width:
            raise OverflowError('溢位')

    def _IntToBoolConv(self, width: int = None, data: int = None) -> list:
        if data is None or width is None:
            data = self._data
            width = self._width
        arr = ([bool(data & (1 << n)) for n in range(width)])
        #arr.reverse()
        return arr

    def _bool2int(self, bool_array: list) -> int:
        ret = 0
        for _ in range(len(bool_array)):
            ret += int(bool_array[_]) << _

        return ret

    def getIntVal(self) -> int:
        return self._data


def cascade(*wires) -> Wire:
    width = 0
    data = 0
    for wire in reversed(wires):
        if not type(wire) == Wire:
            raise TypeError()
        # print(wire.data, wire.width)
        data += wire.data << width
        width += wire.width

    return Wire(data=data, width=width)

In [63]:
# # 範例用法
# signal_a = Wire(width=5, data=31)
# signal_b = Wire(width=2, data=3)
# signal_c = Wire(width=3, data=7)
# signal_d = cascade(signal_a, signal_b, signal_c)
# print('PASS 1>', signal_d)

# signal_a = Wire(width=5, data=0b10101)
# signal_b = signal_a[4:2]
# print('PASS 2>', signal_a, ' >> ', signal_b)

# signal_a = Wire(width=5, data=0b11111)
# signal_b = signal_a[2]
# print('PASS 3>', signal_a,  ' >> ', signal_b)

# signal_a[2:0] = 0b0111
# print('PASS 4>', signal_a)

# signal_a = Wire(width=5, data=0b11000)
# print('PASS 5>', signal_a)


* ### 載入Overlay、設定DMA

In [64]:
# design_name = baseDir + 'top_module.bit'
# ol = Overlay(design_name)
# dma = ol.axi_dma_0
# dma_send = ol.axi_dma_0.sendchannel
# dma_recv = ol.axi_dma_0.recvchannel
# mmio = MMIO(0x4000_0000, 0x1000) # <Not Sure Address Setting> allocate (base address, length)

In [65]:
# dma_send.stop()
# dma_recv.stop()
# dma_send.start()
# dma_recv.start()

In [66]:
# ol?
# ol.ip_dict

In [67]:
# dma.register_map

In [68]:
# dma.buffer_max_size

In [69]:
# dma.description 

## 設定預設值
---

In [70]:
# MAC_NUM                    = 256
# WEIGHT_BRAM_ADDRESS_WIDTH  = 12
# BIAS_BRAM_ADDRESS_WIDTH    = 9
# BIAS_BRAM_DATA_WIDTH       = 32
# AXIS_PRELOAD_FIFO_DEPTH    = 4
# AXIS_FIFO_SIZE             = 16
# M_AXIS_FIFO_DEPTH          = 4
# C_S_AXIS_TDATA_WIDTH       = 32
# C_S_AXI_DATA_WIDTH         = 32
# C_S_AXI_ADDR_WIDTH         = 5
# C_M_AXIS_TDATA_WIDTH       = 32

AXI_0 = Wire(width=32, data=0)
AXI_1 = Wire(width=32, data=0)
AXI_2 = Wire(width=32, data=0)
AXI_3 = Wire(width=32, data=0)
AXI_4 = Wire(width=32, data=0)
AXI_5 = Wire(width=32, data=0)
AXI_6 = Wire(width=32, data=0)
AXI_7 = Wire(width=32, data=0)

## 函數定義
---

* ### Read AXI
[API說明](https://pynq.readthedocs.io/en/v3.0.0/pynq_libraries/mmio.html)
[API文件](https://pynq.readthedocs.io/en/v3.0.0/pynq_package/pynq.mmio.html#pynq-mmio)

In [71]:
def read_AXI(addr):
    ret = mmio.read(addr)
#     ret = mmio.read(addr << 2)
#     print('AXI_Received > ', hex(ret), '(h) = ' , ret , '(d)')  
    return ret

* ### Write AXI
[API說明](https://pynq.readthedocs.io/en/v3.0.0/pynq_libraries/mmio.html)
[API文件](https://pynq.readthedocs.io/en/v3.0.0/pynq_package/pynq.mmio.html#pynq-mmio)

In [72]:
def write_AXI(addr, data):
    mmio.write_reg(addr, data)

* ### Send data via AXIS
[API說明](https://pynq.readthedocs.io/en/v3.0.0/pynq_libraries/dma.html)
[API文件](https://pynq.readthedocs.io/en/v3.0.0/pynq_package/pynq.lib/pynq.lib.dma.html#pynq-lib-dma)

In [73]:
# '''
#     Send AXIS DATA
    
#     input: 
#     - data: int (if size = 1) / list (if size != 1)
#     - size: int, if multiple input
#     output:
#     (none)
    
# '''
def send_AXIS(data, list_size:int = 1) -> None:
    assert list_size > 0 # 長度至少要大於0
#     print('send AXIS start')    

    input_data = allocate(shape=(list_size,),dtype=np.uint32)
    if list_size == 1:
        if (type(data) == int) or (type(data) == np.int32):
            input_data[0] = data
#             print('Send function got 1 data: {:032b}'.format(data))
        elif type(data) == Wire:
            input_data[0] = data.data
#             print('Send function got 1 wire: {}'.format(data))
    else:
        for _ in range(list_size):
            input_data[_] = data[_]
    
#     print(input_data)
    dma_send.transfer(input_data)
    dma_send.wait()
    del input_data
  
    #print('AXIS Data Sent! Data length = {}'.format(list_size))

* ### Read State

In [74]:
def get_bit(data,start,end):
    data = data>>end
    data = data&(pow(2,start-end+1)-1)
    return data

In [75]:
# get_bit(11010,2,0)

In [76]:
# def read_FSM():
#     ret = mmio.read(4 << 2)
#     print('control_load_weight_state      > ', get_bit(ret,4,0))
#     print('control_load_ifmaps_state      > ', get_bit(ret,9,5))
#     print('control_write_weight_state     > ', get_bit(ret,12,10))
#     print('control_write_bias_state       > ', get_bit(ret,15,13))
#     print('control_load_bias_state        > ', get_bit(ret,18,16))
#     print('weight_bram_control_read_state > ', get_bit(ret,20,19))
#     print('weight_bram_control_write_state> ', get_bit(ret,23,21))
#     print('bias_bram_control_read_state   > ', get_bit(ret,25,24))
#     print('bias_bram_control_write_state  > ', get_bit(ret,28,26))
#     return ret

* ### Send Write Weight Start Command

In [77]:
def write_weight_start(AXI_0:Wire):
    AXI_0[7:0] = 12
    
    write_AXI(0,AXI_0.data)
    return AXI_0    

* ### Send Write Bias Command

In [78]:
def write_bias_start(AXI_0:Wire):
    AXI_0[7:0] = 34
    
    write_AXI(0,AXI_0.data)
    return AXI_0   

* ### Send Compute Start Command

In [79]:
def compute_start(AXI_0:Wire):
    AXI_0[7:0] = 87
    
    write_AXI(0,AXI_0.data)
    return AXI_0   

wip

* ### Send Kernel Size

In [80]:
def set_kernel_size(AXI_2:Wire, kernel_size:int = 3):

    if 0 < kernel_size < 6:
        AXI_2[4:0] = 0b00001 << (kernel_size-1)
    else:
        AXI_2[4:0] = 0b00100

#     print(AXI_2[4:0]) # 請務必再三確認這是對的
    write_AXI(8,AXI_2.data)

    return AXI_2    

* ### Set Ofmaps Channel And Input Channel

In [81]:
def set_ofmaps_channel_and_input_channel(AXI_0:Wire,ofmaps_channel:int = 0 , input_channel:int = 0):
    AXI_0[19:8] = input_channel
    AXI_0[31:20] = ofmaps_channel
    
    write_AXI(0,AXI_0.data)
    return AXI_0

* ### Set Stride Function Ofmaps Width

In [82]:
def set_stride_function_ofmaps_width(AXI_1:Wire,stride:int = 0 , func:int = 0 , ofmaps_width:int = 0):
    AXI_1.data = 0
    
    AXI_1[1:0] = func
    AXI_1[10:2] = ofmaps_width
    AXI_1[13:11] = stride
    
    write_AXI(4,AXI_1.data)
    return AXI_1

* ### Write Weight

In [83]:
def write_weight(weight_arr: np.ndarray):
    send_AXIS(weight_arr,weight_arr.size)

* ### Write Bias

In [84]:
def write_bias(bias_arr: np.ndarray):
    send_AXIS(bias_arr,bias_arr.size)

* ### Write Ifmaps

In [85]:
def write_ifmaps(ifmaps_arr: np.ndarray):
#     print(ifmaps_arr.size)
#     print(ifmaps_arr)

    send_AXIS(ifmaps_arr,ifmaps_arr.size)


* ### First layer ifmaps modeling

In [86]:
def first_ifmaps_modeling(file_ifmaps: np.ndarray,weight_width: int,weight_height: int,ifmaps_width: int,ifmaps_height: int,
                          ifmaps_channel: int,stride: int):
    

    o_hight = int(((ifmaps_height - weight_height) / stride + 1))
    o_channel = (ifmaps_channel+31)//32
    
#     pre_modeling_arr = np.empty((o_channel,ifmaps_height,ifmaps_width),dtype = np.uint32)

#     for h in range(0,ifmaps_height,stride):
#         for w in range(ifmaps_width):
#             axis_tmp = 0
#             idx = 0 
#             for ch in range(ifmaps_channel):
#                 axis_tmp |= file_ifmaps[ch][h][w]<<idx;
#                 idx = idx + 1
                
#                 if((ch % 32 == 31) or (ch == ifmaps_channel-1)):
#                     pre_modeling_arr[ch//32][h][w] = axis_tmp
#                     axis_tmp = 0 
#                     idx = 0
    # 取得陣列的指標
    input_arr_ptr = file_ifmaps.ctypes.data_as(POINTER(c_uint))

    # 呼叫 SO 檔案中的 sort 函數
    ret_ptr = ifmaps_premodeling_dll.pre_modeling(input_arr_ptr, c_uint(ifmaps_height),c_uint(ifmaps_width)
                                                  ,c_uint(ifmaps_channel),c_uint(stride),c_uint(weight_height))

    # 將回傳指標指定大小及型態  [0]<-不懂用意
    ret_arr_size = ifmaps_height * ifmaps_width * o_channel
    ret_ptr_cast = ctypes.cast(ret_ptr, ctypes.POINTER(ctypes.c_uint*ret_arr_size))[0]

    # 將buffer轉換成numpy array
    pre_modeling_arr = np.frombuffer(ret_ptr_cast, np.uint32)
    
    # 將原ret_ptr的陣列釋放
#     ifmaps_premodeling_dll.free_mem(ret_ptr)
    
#     pre_modeling_arr = pre_modeling_arr.reshape((o_channel,ifmaps_height,ifmaps_width))
    
     
#     AXIS_arr = []
#     AXIS_arr = np.array(AXIS_arr).astype(np.uint32)

#     for ofmaps_h in range(0,o_hight,stride):
#         tmp_arr = pre_modeling_arr[:, ofmaps_h:(ofmaps_h+weight_height),:]
#         tmp_arr = tmp_arr.transpose(2,1,0)
#         tmp_arr = tmp_arr.reshape(-1)
#         AXIS_arr = np.concatenate((AXIS_arr,tmp_arr),dtype=np.uint32)
    
    # 取得陣列的指標
    pre_modeling_arr_ptr = pre_modeling_arr.ctypes.data_as(POINTER(c_uint))

    # 呼叫 SO 檔案中的 sort 函數
    ret_ptr = arr_axis_modeling_dll.arr_axis_modeling(pre_modeling_arr_ptr,
                                                      c_uint(ifmaps_height),c_uint(ifmaps_width),c_uint(o_channel),
                                                      c_uint(stride),c_uint(weight_height))

    # 將回傳指標指定大小及型態  [0]<-不懂用意
    ret_arr_size = o_hight * ifmaps_width * o_channel * weight_height
    ret_ptr_cast = ctypes.cast(ret_ptr, ctypes.POINTER(ctypes.c_int32*ret_arr_size))[0]

    # 將buffer轉換成numpy array
    AXIS_arr = np.frombuffer(ret_ptr_cast, np.uint32)
    
    # 將原ret_ptr的陣列釋放
#     arr_axis_modeling_dll.free_mem(ret_ptr)
   

    return AXIS_arr

In [87]:
# array_from_front_end = np.loadtxt('ifmaps0.txt', dtype=np.uint32)
# ifmaps_arr = first_ifmaps_modeling(array_from_front_end,setting_kernel_size[0],setting_kernel_size[0],
#                                    setting_ifmaps_width[0],setting_ifmaps_width[0],setting_input_channel[0],
#                                    setting_stride[0])
# print(ifmaps_arr)

* ### ofmaps_padding

In [88]:
def ofmaps_padding(ifmaps_in):
    if(ifmaps_in.shape[2]%2==0):
        h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.uint32)
        h_padding[:,1::2].fill(0xffff_ffff)
        ifmaps_in = np.concatenate((h_padding,ifmaps_in), axis=2) 
        h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.uint32)
        h_padding[:,::2].fill(0xffff_ffff)
        ifmaps_in = np.concatenate((ifmaps_in,h_padding), axis=2) 
    else:
        h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.uint32)
        h_padding[:,1::2].fill(0xffff_ffff)
        ifmaps_in = np.concatenate((h_padding,ifmaps_in,h_padding), axis=2) 

    if(ifmaps_in.shape[1]%2==0):
        v_padding = np.zeros((ifmaps_in.shape[0],1,ifmaps_in.shape[2]),dtype=np.uint32)
        v_padding[:,:,::2].fill(0xffff_ffff)
        ifmaps_in = np.concatenate((v_padding,ifmaps_in), axis=1) 
        v_padding = np.concatenate((v_padding[:,:,1:],v_padding[:,:,:1]), axis=2)
        ifmaps_in = np.concatenate((ifmaps_in,v_padding), axis=1) 
    else:
        v_padding = np.zeros((ifmaps_in.shape[0],1,ifmaps_in.shape[2]),dtype=np.uint32)
        v_padding[:,:,::2].fill(0xffff_ffff)
        ifmaps_in = np.concatenate((v_padding,ifmaps_in, v_padding), axis=1) 

    return ifmaps_in

* ### ifmaps modeling

In [89]:
def ifmaps_modeling(ifmaps_in,ifmaps_width,ifmaps_height,ifmaps_channel,weight_width,weight_height,stride,padding=True):

    o_channel = (ifmaps_channel+31)//32
    ifmaps_in = ifmaps_in.reshape(ifmaps_height,ifmaps_width,o_channel)
    ifmaps_in = ifmaps_in.transpose(2,0,1)
    
    if padding == True:
        
        ifmaps_in = ofmaps_padding(ifmaps_in)

        o_hight = int(((ifmaps_in.shape[2] - weight_width) / stride + 1))
        o_width = ifmaps_width
        
        ifmaps_width = ifmaps_width+2
        ifmaps_height = ifmaps_height+2
        
    else:
        if (ifmaps_width%stride!=0):
            pad_width = ifmaps_width%stride
            h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],pad_width),dtype=np.uint32)
            ifmaps_in = np.concatenate((ifmaps_in,h_padding), axis=2) 
            v_padding = np.zeros((ifmaps_in.shape[0],pad_width,ifmaps_in.shape[2]),dtype=np.uint32)
            ifmaps_in = np.concatenate((ifmaps_in,v_padding), axis=1) 
            ifmaps_height = ifmaps_height+pad_width
            ifmaps_width = ifmaps_width+pad_width
        o_hight = ifmaps_height
        o_width = ifmaps_width
        
#     start_time = time.time()
#     for ofmaps_h in range(0,o_hight,stride):
#         for ifmaps_w in range(ifmaps_in.shape[2]):
#             for height in range(weight_height):
#                 for ch in range(o_channel):
#                     out_arr = np.concatenate((out_arr,[ifmaps_in[ch][ofmaps_h+height][ifmaps_w]]),dtype=np.uint32)
# #                     out_arr = np.append(out_arr,ifmaps_in[ch][ofmaps_h+height][ifmaps_w])
#     end_time = time.time()
#     print("執行時間：%f 秒" %(end_time-start_time))
#     
    
#     #start_time = time.time()
#     out_arr = []
#     out_arr = np.array(out_arr).astype(np.uint32)
#     for ofmaps_h in range(0,o_hight,stride):
#         tmp_arr = ifmaps_in[:, ofmaps_h:(ofmaps_h+weight_height),:]
#         tmp_arr = tmp_arr.transpose(2,1,0)
#         tmp_arr = tmp_arr.reshape(-1)
#         out_arr = np.concatenate((out_arr,tmp_arr),dtype=np.uint32)

#     # 取得陣列的指標
    ifmaps_in = ifmaps_in.reshape(-1)
    ifmaps_in_ptr = ifmaps_in.ctypes.data_as(POINTER(c_uint))

    # 呼叫 SO 檔案中的 sort 函數
    ret_ptr = arr_axis_modeling_dll.arr_axis_modeling(ifmaps_in_ptr,
                                                      c_uint(ifmaps_height),c_uint(ifmaps_width),c_uint(o_channel),
                                                      c_uint(stride),c_uint(weight_height))

    # 將回傳指標指定大小及型態  [0]<-不懂用意
    ret_arr_size = o_hight * ifmaps_width * o_channel * weight_height // stride
    ret_ptr_cast = ctypes.cast(ret_ptr, ctypes.POINTER(ctypes.c_int32*ret_arr_size))[0]

    # 將buffer轉換成numpy array
    out_arr = np.frombuffer(ret_ptr_cast, np.uint32)
    
    #將原ret_ptr的陣列釋放
#     arr_axis_modeling_dll.free_mem(ret_ptr)
    
#     end_time = time.time()
#     print(end_time-start_time)
#     print(out_arr)
    
#     print(out_arr.size)
    return out_arr

* ### 驗證AXIS與ANS是否相同

In [90]:
# def validate_with_ans(ans_file_dir):
#     out_arr = []
#     out_arr = np.array(out_arr).astype(int)
    
#     with open(ans_file_dir, 'r') as f:
#         hex_data = f.read()
#     hex_split = hex_data.split()

#     for _ in range(len(hex_split)):
#         out_arr = np.append(out_arr,int(hex_split[_], 16))     
    
#     for _ in range(out_arr.size):
#         if(out_arr[_] != output_data[_]):
#             return 1
#     return 0 

## 主程式起點
---

In [91]:
design_name = 'baseDir + top_module.bit'
ol = Overlay(design_name)
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel
mmio = MMIO(0x4000_0000, 0x1000) # <Not Sure Address Setting> allocate (base address, length)

In [92]:
####CIFAR10
# setting_kernel_size    = [3,3,2,3,3,2,3,3,2]
# setting_input_channel  = [3,128,256,256,256,256,256,128,128]
# setting_ofmaps_channel = [128,256,256,256,256,256,128,128,128]
# setting_stride         = [1,1,2,1,1,2,1,1,2]
# setting_function       = [0,0,1,0,0,1,0,0,1]
# setting_ifmaps_width   = [34,32,32,16,16,16,8,8,8]
# setting_ofmaps_width   = [32,32,16,16,16,8,8,8,4]
# setting_out_class      = 10
# setting_use_fcbias     = 1  # 1: Ture  0: False  
# label_name = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# setting_output_array_size = [4096,8192,2048,2048,2048,512,256,256,64]

In [93]:
####CIFAR10
# weight_npy = {}

# weight_npy["weight0_npy"] = np.array(np.load('weight0.npy'))
# weight_npy["weight1_npy"] = np.array(np.load('weight1.npy'))
# weight_npy["weight3_npy"] = np.array(np.load('weight3.npy'))
# weight_npy["weight4_npy"] = np.array(np.load('weight4.npy'))
# weight_npy["weight6_npy"] = np.array(np.load('weight6.npy'))
# weight_npy["weight7_npy"] = np.array(np.load('weight7.npy'))
# weight_npy["fc_weight_npy"] = np.array(np.load('fc_weight.npy'))

# bias_npy = {}

# bias_npy["bias0_npy"] = np.array(np.load('bias0.npy'))
# bias_npy["bias1_npy"] = np.array(np.load('bias1.npy'))
# bias_npy["bias3_npy"] = np.array(np.load('bias3.npy'))
# bias_npy["bias4_npy"] = np.array(np.load('bias4.npy'))
# bias_npy["bias6_npy"] = np.array(np.load('bias6.npy'))
# bias_npy["bias7_npy"] = np.array(np.load('bias7.npy'))
# bias_npy["fc_bias_npy"] = np.array(np.load('fc_bias.npy'))


In [94]:
####MNIST
setting_kernel_size    = [3,2,3,2,3,2]
setting_input_channel  = [1,32,32,64,64,128]
setting_ofmaps_channel = [32,32,64,64,128,128]
setting_stride         = [1,2,1,2,1,2]
setting_function       = [0,1,0,1,0,1]
setting_ifmaps_width   = [30,28,14,14,7,7]
setting_ofmaps_width   = [28,14,14,7,7,4]
setting_out_class      = 10
setting_use_fcbias     = 1  # 1: Ture  0: False  
label_name = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

setting_output_array_size = [784,196,392,98,196,64]


In [95]:
####MNIST
weight_npy = {}

weight_npy["weight0_npy"] = np.array(np.load('weight0.npy'))
weight_npy["weight2_npy"] = np.array(np.load('weight2.npy'))
weight_npy["weight4_npy"] = np.array(np.load('weight4.npy'))
weight_npy["fc_weight_npy"] = np.array(np.load('fc_weight.npy'))

bias_npy = {}

bias_npy["bias0_npy"] = np.array(np.load('bias0.npy'))
bias_npy["bias2_npy"] = np.array(np.load('bias2.npy'))
bias_npy["bias4_npy"] = np.array(np.load('bias4.npy'))
bias_npy["fc_bias_npy"] = np.array(np.load('fc_bias.npy'))


In [96]:
# array_from_front_end = np.loadtxt('ifmaps0.txt', dtype=np.uint32)

In [97]:
output_data = allocate(shape = (max(setting_output_array_size), ),dtype=np.uint32)
fc_weight_ptr = weight_npy["fc_weight_npy"].ctypes.data_as(POINTER(c_float))
fc_bias_ptr = bias_npy["fc_bias_npy"].ctypes.data_as(POINTER(c_float))

In [102]:
class main_function:
    def __init__ (self,array_from_front_end):
        design_name = baseDir + 'top_module.bit'
        ol = Overlay(design_name)
        dma = ol.axi_dma_0
        dma_send = ol.axi_dma_0.sendchannel
        dma_recv = ol.axi_dma_0.recvchannel
        mmio = MMIO(0x4000_0000, 0x1000) # <Not Sure Address Setting> allocate (base address, length)
        #######################
        setting_kernel_size    = [3,2,3,2,3,2]
        setting_input_channel  = [1,32,32,64,64,128]
        setting_ofmaps_channel = [32,32,64,64,128,128]
        setting_stride         = [1,2,1,2,1,2]
        setting_function       = [0,1,0,1,0,1]
        setting_ifmaps_width   = [30,28,14,14,7,7]
        setting_ofmaps_width   = [28,14,14,7,7,4]
        setting_out_class      = 10
        setting_use_fcbias     = 1  # 1: Ture  0: False  
        label_name = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
        setting_output_array_size = [784,196,392,98,196,64]
        ############################
        ####MNIST
        weight_npy = {}

        weight_npy["weight0_npy"] = np.array(np.load('weight0.npy'))
        weight_npy["weight2_npy"] = np.array(np.load('weight2.npy'))
        weight_npy["weight4_npy"] = np.array(np.load('weight4.npy'))
        weight_npy["fc_weight_npy"] = np.array(np.load('fc_weight.npy'))

        bias_npy = {}

        bias_npy["bias0_npy"] = np.array(np.load('bias0.npy'))
        bias_npy["bias2_npy"] = np.array(np.load('bias2.npy'))
        bias_npy["bias4_npy"] = np.array(np.load('bias4.npy'))
        bias_npy["fc_bias_npy"] = np.array(np.load('fc_bias.npy'))
        
        output_data = allocate(shape = (max(setting_output_array_size), ),dtype=np.uint32)
        fc_weight_ptr = weight_npy["fc_weight_npy"].ctypes.data_as(POINTER(c_float))
        fc_bias_ptr = bias_npy["fc_bias_npy"].ctypes.data_as(POINTER(c_float))
        ##############################
        AXI_0 = Wire(width=32, data=0)
        AXI_1 = Wire(width=32, data=0)
        AXI_2 = Wire(width=32, data=0)
        AXI_3 = Wire(width=32, data=0)
        AXI_4 = Wire(width=32, data=0)
        AXI_5 = Wire(width=32, data=0)
        AXI_6 = Wire(width=32, data=0)
        AXI_7 = Wire(width=32, data=0)
        array_from_front_end = np.array(array_from_front_end)
        load_weight_bias_times = 0
        ifmaps_modeling_times = 0
        PL_times = 0
        
    def set_kernel_size(self, AXI_2:Wire, kernel_size:int = 3):

        if 0 < kernel_size < 6:
            self.AXI_2[4:0] = 0b00001 << (kernel_size-1)
        else:
            self.AXI_2[4:0] = 0b00100

    #     print(AXI_2[4:0]) # 請務必再三確認這是對的
        self.write_AXI(8,AXI_2.data)

        return AXI_2
    
    def set_ofmaps_channel_and_input_channel(self, AXI_0:Wire,ofmaps_channel:int = 0 , input_channel:int = 0):
        self.AXI_0[19:8] = input_channel
        self.AXI_0[31:20] = ofmaps_channel

        self.write_AXI(0,AXI_0.data)
        return AXI_0
    
    def set_stride_function_ofmaps_width(self, AXI_1:Wire,stride:int = 0 , func:int = 0 , ofmaps_width:int = 0):
        AXI_1.data = 0

        AXI_1[1:0] = func
        AXI_1[10:2] = ofmaps_width
        AXI_1[13:11] = stride

        write_AXI(4,AXI_1.data)
        return AXI_1
    
    def send_AXIS(self, data, list_size:int = 1) -> None:
        assert list_size > 0 # 長度至少要大於0
    #     print('send AXIS start')    

        input_data = allocate(shape=(list_size,),dtype=np.uint32)
        if list_size == 1:
            if (type(data) == int) or (type(data) == np.int32):
                input_data[0] = data
    #             print('Send function got 1 data: {:032b}'.format(data))
            elif type(data) == Wire:
                input_data[0] = data.data
    #             print('Send function got 1 wire: {}'.format(data))
        else:
            for _ in range(list_size):
                input_data[_] = data[_]

    #     print(input_data)
        dma_send.transfer(input_data)
        dma_send.wait()
        del input_data

        #print('AXIS Data Sent! Data length = {}'.format(list_size))
    
    def write_weight_start(self, AXI_0:Wire):
        AXI_0[7:0] = 12

        write_AXI(0,AXI_0.data)
        return AXI_0
    
    def write_weight(self, weight_arr: np.ndarray):
        send_AXIS(weight_arr,weight_arr.size)
    
    def write_bias_start(self, AXI_0:Wire):
        AXI_0[7:0] = 34

        write_AXI(0,AXI_0.data)
        return AXI_0
    
    def write_bias(self, bias_arr: np.ndarray):
        send_AXIS(bias_arr,bias_arr.size)
        
    def write_AXI(self, addr, data):
        mmio.write_reg(addr, data)
    
    def read_AXI(self, addr):
        ret = mmio.read(addr)
    #     ret = mmio.read(addr << 2)
    #     print('AXI_Received > ', hex(ret), '(h) = ' , ret , '(d)')  
        return ret
    
    def compute_start(self, AXI_0:Wire):
        AXI_0[7:0] = 87

        write_AXI(0,AXI_0.data)
        return AXI_0
    
    def write_ifmaps(self, ifmaps_arr: np.ndarray):
    #     print(ifmaps_arr.size)
    #     print(ifmaps_arr)

        send_AXIS(ifmaps_arr,ifmaps_arr.size)

    
    def first_ifmaps_modeling(self, file_ifmaps: np.ndarray,weight_width: int,weight_height: int,ifmaps_width: int,ifmaps_height: int,
                          ifmaps_channel: int,stride: int):

        o_hight = int(((ifmaps_height - weight_height) / stride + 1))
        o_channel = (ifmaps_channel+31)//32

    #     pre_modeling_arr = np.empty((o_channel,ifmaps_height,ifmaps_width),dtype = np.uint32)

    #     for h in range(0,ifmaps_height,stride):
    #         for w in range(ifmaps_width):
    #             axis_tmp = 0
    #             idx = 0 
    #             for ch in range(ifmaps_channel):
    #                 axis_tmp |= file_ifmaps[ch][h][w]<<idx;
    #                 idx = idx + 1

    #                 if((ch % 32 == 31) or (ch == ifmaps_channel-1)):
    #                     pre_modeling_arr[ch//32][h][w] = axis_tmp
    #                     axis_tmp = 0 
    #                     idx = 0
        # 取得陣列的指標
        input_arr_ptr = file_ifmaps.ctypes.data_as(POINTER(c_uint))

        # 呼叫 SO 檔案中的 sort 函數
        ret_ptr = ifmaps_premodeling_dll.pre_modeling(input_arr_ptr, c_uint(ifmaps_height),c_uint(ifmaps_width)
                                                      ,c_uint(ifmaps_channel),c_uint(stride),c_uint(weight_height))

        # 將回傳指標指定大小及型態  [0]<-不懂用意
        ret_arr_size = ifmaps_height * ifmaps_width * o_channel
        ret_ptr_cast = ctypes.cast(ret_ptr, ctypes.POINTER(ctypes.c_uint*ret_arr_size))[0]

        # 將buffer轉換成numpy array
        pre_modeling_arr = np.frombuffer(ret_ptr_cast, np.uint32)

        # 將原ret_ptr的陣列釋放
    #     ifmaps_premodeling_dll.free_mem(ret_ptr)

    #     pre_modeling_arr = pre_modeling_arr.reshape((o_channel,ifmaps_height,ifmaps_width))


    #     AXIS_arr = []
    #     AXIS_arr = np.array(AXIS_arr).astype(np.uint32)

    #     for ofmaps_h in range(0,o_hight,stride):
    #         tmp_arr = pre_modeling_arr[:, ofmaps_h:(ofmaps_h+weight_height),:]
    #         tmp_arr = tmp_arr.transpose(2,1,0)
    #         tmp_arr = tmp_arr.reshape(-1)
    #         AXIS_arr = np.concatenate((AXIS_arr,tmp_arr),dtype=np.uint32)

        # 取得陣列的指標
        pre_modeling_arr_ptr = pre_modeling_arr.ctypes.data_as(POINTER(c_uint))

        # 呼叫 SO 檔案中的 sort 函數
        ret_ptr = arr_axis_modeling_dll.arr_axis_modeling(pre_modeling_arr_ptr,
                                                          c_uint(ifmaps_height),c_uint(ifmaps_width),c_uint(o_channel),
                                                          c_uint(stride),c_uint(weight_height))

        # 將回傳指標指定大小及型態  [0]<-不懂用意
        ret_arr_size = o_hight * ifmaps_width * o_channel * weight_height
        ret_ptr_cast = ctypes.cast(ret_ptr, ctypes.POINTER(ctypes.c_int32*ret_arr_size))[0]

        # 將buffer轉換成numpy array
        AXIS_arr = np.frombuffer(ret_ptr_cast, np.uint32)

        # 將原ret_ptr的陣列釋放
    #     arr_axis_modeling_dll.free_mem(ret_ptr)


        return AXIS_arr
    
    def ofmaps_padding(self, ifmaps_in):
        if(ifmaps_in.shape[2]%2==0):
            h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.uint32)
            h_padding[:,1::2].fill(0xffff_ffff)
            ifmaps_in = np.concatenate((h_padding,ifmaps_in), axis=2) 
            h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.uint32)
            h_padding[:,::2].fill(0xffff_ffff)
            ifmaps_in = np.concatenate((ifmaps_in,h_padding), axis=2) 
        else:
            h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.uint32)
            h_padding[:,1::2].fill(0xffff_ffff)
            ifmaps_in = np.concatenate((h_padding,ifmaps_in,h_padding), axis=2) 

        if(ifmaps_in.shape[1]%2==0):
            v_padding = np.zeros((ifmaps_in.shape[0],1,ifmaps_in.shape[2]),dtype=np.uint32)
            v_padding[:,:,::2].fill(0xffff_ffff)
            ifmaps_in = np.concatenate((v_padding,ifmaps_in), axis=1) 
            v_padding = np.concatenate((v_padding[:,:,1:],v_padding[:,:,:1]), axis=2)
            ifmaps_in = np.concatenate((ifmaps_in,v_padding), axis=1) 
        else:
            v_padding = np.zeros((ifmaps_in.shape[0],1,ifmaps_in.shape[2]),dtype=np.uint32)
            v_padding[:,:,::2].fill(0xffff_ffff)
            ifmaps_in = np.concatenate((v_padding,ifmaps_in, v_padding), axis=1) 

        return ifmaps_in
    
    def ifmaps_modeling(self, ifmaps_in,ifmaps_width,ifmaps_height,ifmaps_channel,weight_width,weight_height,stride,padding=True):

        o_channel = (ifmaps_channel+31)//32
        ifmaps_in = ifmaps_in.reshape(ifmaps_height,ifmaps_width,o_channel)
        ifmaps_in = ifmaps_in.transpose(2,0,1)

        if padding == True:

            ifmaps_in = ofmaps_padding(ifmaps_in)

            o_hight = int(((ifmaps_in.shape[2] - weight_width) / stride + 1))
            o_width = ifmaps_width

            ifmaps_width = ifmaps_width+2
            ifmaps_height = ifmaps_height+2

        else:
            if (ifmaps_width%stride!=0):
                pad_width = ifmaps_width%stride
                h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],pad_width),dtype=np.uint32)
                ifmaps_in = np.concatenate((ifmaps_in,h_padding), axis=2) 
                v_padding = np.zeros((ifmaps_in.shape[0],pad_width,ifmaps_in.shape[2]),dtype=np.uint32)
                ifmaps_in = np.concatenate((ifmaps_in,v_padding), axis=1) 
                ifmaps_height = ifmaps_height+pad_width
                ifmaps_width = ifmaps_width+pad_width
            o_hight = ifmaps_height
            o_width = ifmaps_width

    #     start_time = time.time()
    #     for ofmaps_h in range(0,o_hight,stride):
    #         for ifmaps_w in range(ifmaps_in.shape[2]):
    #             for height in range(weight_height):
    #                 for ch in range(o_channel):
    #                     out_arr = np.concatenate((out_arr,[ifmaps_in[ch][ofmaps_h+height][ifmaps_w]]),dtype=np.uint32)
    # #                     out_arr = np.append(out_arr,ifmaps_in[ch][ofmaps_h+height][ifmaps_w])
    #     end_time = time.time()
    #     print("執行時間：%f 秒" %(end_time-start_time))
    #     

    #     #start_time = time.time()
    #     out_arr = []
    #     out_arr = np.array(out_arr).astype(np.uint32)
    #     for ofmaps_h in range(0,o_hight,stride):
    #         tmp_arr = ifmaps_in[:, ofmaps_h:(ofmaps_h+weight_height),:]
    #         tmp_arr = tmp_arr.transpose(2,1,0)
    #         tmp_arr = tmp_arr.reshape(-1)
    #         out_arr = np.concatenate((out_arr,tmp_arr),dtype=np.uint32)

    #     # 取得陣列的指標
        ifmaps_in = ifmaps_in.reshape(-1)
        ifmaps_in_ptr = ifmaps_in.ctypes.data_as(POINTER(c_uint))

        # 呼叫 SO 檔案中的 sort 函數
        ret_ptr = arr_axis_modeling_dll.arr_axis_modeling(ifmaps_in_ptr,
                                                          c_uint(ifmaps_height),c_uint(ifmaps_width),c_uint(o_channel),
                                                          c_uint(stride),c_uint(weight_height))

        # 將回傳指標指定大小及型態  [0]<-不懂用意
        ret_arr_size = o_hight * ifmaps_width * o_channel * weight_height // stride
        ret_ptr_cast = ctypes.cast(ret_ptr, ctypes.POINTER(ctypes.c_int32*ret_arr_size))[0]

        # 將buffer轉換成numpy array
        out_arr = np.frombuffer(ret_ptr_cast, np.uint32)

        #將原ret_ptr的陣列釋放
    #     arr_axis_modeling_dll.free_mem(ret_ptr)

    #     end_time = time.time()
    #     print(end_time-start_time)
    #     print(out_arr)

    #     print(out_arr.size)
        return out_arr
    
    total_start_time = time.time()

    for now_layer in range(6):
        AXI_2 = self.set_kernel_size(self.AXI_2,self.setting_kernel_size[now_layer])  
        AXI_0 = self.set_ofmaps_channel_and_input_channel(self.AXI_0,setting_ofmaps_channel[now_layer],self.setting_input_channel[now_layer])
        AXI_1 = self.set_stride_function_ofmaps_width(self.AXI_1,setting_stride[now_layer],self.setting_function[now_layer],
                                                      self.setting_ofmaps_width[now_layer])         

        dma_send.start()
        dma_recv.start()

        if(setting_function[now_layer] == 0):

    #         load_weight_bias_start_time = time.time()

            AXI_0 = write_weight_start(AXI_0) 
            write_weight(weight_npy["weight{}_npy".format(now_layer)])

            while(read_AXI(12) != 0x1):
                pass

            AXI_0 = write_bias_start(AXI_0)
            write_bias(bias_npy["bias{}_npy".format(now_layer)])


            while(read_AXI(12) != 0x2):
                pass

    #         load_weight_bias_end_time = time.time()
    #         load_weight_bias_times = load_weight_bias_times + (load_weight_bias_end_time-load_weight_bias_start_time)
    #         print("load_weight_bias 執行時間：%f 秒" %(load_weight_bias_end_time-load_weight_bias_start_time))


        AXI_0 = compute_start(AXI_0)

        if(now_layer == 0):

    #         first_ifmaps_modeling_start_time = time.time()

            ifmaps_arr = first_ifmaps_modeling(array_from_front_end,setting_kernel_size[0],setting_kernel_size[0],
                                               setting_ifmaps_width[0],setting_ifmaps_width[0],setting_input_channel[0],
                                               setting_stride[0])
    #         first_ifmaps_modeling_end_time = time.time()
    #         ifmaps_modeling_times = ifmaps_modeling_times + (first_ifmaps_modeling_end_time-first_ifmaps_modeling_start_time)
    #         print("first_ifmaps_modeling 執行時間：%f 秒" %(first_ifmaps_modeling_end_time-first_ifmaps_modeling_start_time))
        else:
    #         ifmaps_modeling_start_time = time.time()
            if(setting_function[now_layer] == 0):
                padding_en = True
            else:
                padding_en = False
    #         ifmaps_arr = ifmaps_modeling(output_data,setting_ifmaps_width[now_layer],setting_ifmaps_width[now_layer],
    #                                      setting_input_channel[now_layer],setting_kernel_size[now_layer],
    #                                      setting_kernel_size[now_layer],setting_stride[now_layer],padding_en)

            ifmaps_arr = ifmaps_modeling(output_data[:setting_output_array_size[now_layer-1]],setting_ifmaps_width[now_layer],
                                         setting_ifmaps_width[now_layer],setting_input_channel[now_layer],
                                         setting_kernel_size[now_layer],setting_kernel_size[now_layer],
                                         setting_stride[now_layer],padding_en)

    #         ifmaps_modeling_end_time = time.time()
    #         ifmaps_modeling_times = ifmaps_modeling_times + (ifmaps_modeling_end_time-ifmaps_modeling_start_time)
    #         print("ifmaps_modeling 執行時間：%f 秒" %(ifmaps_modeling_end_time-ifmaps_modeling_start_time))

        PL_start_time = time.time()
    #     output_data = allocate(shape = (setting_output_array_size[now_layer], ),dtype=np.uint32)
        dma_recv.transfer(output_data)
        write_ifmaps(ifmaps_arr)
        dma_recv.wait()

        PL_end_time = time.time()
        PL_times = PL_times + (PL_end_time-PL_start_time)
    #     print("PL 執行時間：%f 秒" %(PL_end_time-PL_start_time))


        while(read_AXI(12) != 0xffff_ffff):
            pass

    #     validate_ret = validate_with_ans("./AXIS_ANS{}.txt".format(now_layer))
    #     if(validate_ret == 0):
    #         print("validattion success")
    #     else:
    #         print("validattion fail")
    #     np.savetxt('PL_out{}.txt'.format(now_layer), output_data, newline='\n', fmt='%x')

        dma_send.stop()
        dma_recv.stop()   

    conv_end_time = time.time()
    # 呼叫 SO 檔案中的 FC 函數
    fc_start_time = time.time()
    output_data_ptr = output_data.ctypes.data_as(POINTER(c_uint))  # TODO: 這個拖慢了時間 占了 0.001多秒
    answer = arr_axis_modeling_dll.arr_axis_fc_modeling(output_data_ptr,
                                                        setting_ofmaps_channel[5]*setting_ofmaps_width[5]*setting_ofmaps_width[5],
                                                        setting_out_class, setting_ofmaps_width[5],
                                                        setting_use_fcbias,
                                                        fc_weight_ptr,fc_bias_ptr)

    # cast_time = time.time()
    # answer_reg = ctypes.cast(answer_ptr, ctypes.POINTER(ctypes.c_float*setting_out_class))[0]
    # out_arr_np = np.frombuffer(answer_reg, np.float32)

    total_end_time = time.time()
    # correct_ans = np.argmax(out_arr_np)
    # argmax_time = time.time()

    print("此張MNIST圖片是:{}".format(label_name[answer]))
    print("PL總執行時間：%f 秒" %(PL_times))
    print("CONV執行時間：%f 秒" %(conv_end_time-total_start_time))
    print("FC執行時間：%f 秒" %(total_end_time-fc_start_time))
    # print("cast執行時間：%f 秒" %(total_end_time-cast_time))
    # print("np.argmaxs執行時間：%f 秒" %(argmax_time-total_end_time))
    print("總執行時間：%f 秒" %(total_end_time-total_start_time))
    # how_many_free_mem()
    # print("load_weight_bias總執行時間：%f 秒" %(load_weight_bias_times))
    # print("ifmaps_modeling總執行時間：%f 秒" %(ifmaps_modeling_times))


此張MNIST圖片是:1
PL總執行時間：0.065482 秒
CONV執行時間：0.074186 秒
FC執行時間：0.000633 秒
總執行時間：0.074825 秒


In [99]:
# for _ in range(setting_output_array_size[5]):
#     print(hex(output_data[_]))

In [100]:
# validate_ret = validate_with_ans("./AXIS_ANS0.txt")
# print(validate_ret)

In [101]:
main_function(np.loadtxt('ifmaps0.txt', dtype=np.uint32))

UnboundLocalError: local variable 'AXI_2' referenced before assignment

## 主程式結束
---

In [ ]:
# read_FSM()

In [ ]:
# dma.register_map

In [ ]:
# print("------------0----------")
# print(AXI_0)
# read_AXI(0)
# print("------------1----------")
# print(AXI_1)
# read_AXI(4)
# print("------------2----------")
# print(AXI_2)
# read_AXI(8)
# print("------------3----------")
# print(AXI_3)
# read_AXI(12)
# print("------------4----------")
# print(AXI_4)
# read_AXI(16)
# print("------------5----------")
# print(AXI_5)
# read_AXI(20)
# print("------------6----------")
# print(AXI_6)
# read_AXI(24)
# print("------------7----------")
# print(AXI_7)
# read_AXI(28)

In [ ]:
# print(dma_send.idle)
# print(dma_send.error)
# print(dma_send.running)
# print(dma_recv.idle)
# print(dma_recv.error)
# print(dma_recv.running)

* ### Calling C Function in Python

In [ ]:
# import platform
# print(platform.architecture())

In [ ]:
# import ctypes
# from ctypes import cdll, c_int32, c_uint, POINTER

# # 載入 SO 動態連結檔案
# mydll = cdll.LoadLibrary('./test.so')

# # 設定 SO 檔案中 sort 函數的參數資料型態
# mydll.array_add_one.argtypes = [POINTER(c_int32), c_uint]

# # 設定 SO 檔案中 sort 函數的傳回值資料型態
# mydll.array_add_one.restype = POINTER(c_int32)

# # 建立 NumPy 資料陣列
# data = np.array([1,2,3,4,5,6,7,8,9])

# # 取得陣列的指標
# dataPtr = data.ctypes.data_as(POINTER(c_int32))

# # 呼叫 SO 檔案中的 sort 函數

# ret_ptr = mydll.array_add_one(dataPtr, data.size)

# # 將回傳指標指定大小及型態  [0]<-不懂用意
# ret_ptr = ctypes.cast(ret_ptr, ctypes.POINTER(ctypes.c_int32*9))[0]

# # 將buffer轉換成numpy array
# ret_arr = np.frombuffer(ret_ptr, np.int32)

# print(ret_arr)
